# Adaptive Melliguer controller

## Setup and imports

In [ ]:
import time
import numpy as np
import mujoco
from mujoco import viewer


# ============================================================
# 1) Utilidades pequeñas
# ============================================================

def clamp(x, lo, hi):
    """Limita un escalar entre [lo, hi]."""
    return max(lo, min(hi, x))

def vee(M):
    """
    Operador vee para matrices skew-symmetric:
    [  0  -z   y
       z   0  -x
      -y   x   0 ]  -> [x, y, z]
    """
    return np.array([M[2, 1], M[0, 2], M[1, 0]])

def quat_to_rotmat(q):
    """Quaternion (w,x,y,z) -> matriz de rotación 3x3."""
    w, x, y, z = q
    R = np.array([
        [1 - 2*(y*y + z*z),     2*(x*y - z*w),     2*(x*z + y*w)],
        [    2*(x*y + z*w), 1 - 2*(x*x + z*z),     2*(y*z - x*w)],
        [    2*(x*z - y*w),     2*(y*z + x*w), 1 - 2*(x*x + y*y)]
    ])
    return R


# ============================================================
# 2) Fuerza desconocida REAL (solo para simular el "mundo")
#    -> el controlador NO la conoce, solo la estima.
# ============================================================

def unknown_force_world(pos):
    """
    Fuerza externa desconocida (en mundo) como función de la posición.
    Cambia esto a gusto. Unidades aprox: Newtons.
    """
    x, y, z = pos
    fx = 1.0 * np.sin(0.8 * x)
    fy = 0.5 * np.cos(0.6 * y)
    fz = 0.8 * np.sin(0.5 * z)
    return np.array([fx, fy, fz])
    # retornar todas las fuerzas cero sin ningun efecto
    # return np.array([0.0, 0.0, 0.0])


# ============================================================
# 3) Trayectoria deseada
# ============================================================

def desired_trajectory(t, p0):
    """
    Retorna: p_d, v_d, a_d, yaw_d, yawdot_d
    (hover con pequeña excitación para que el RLS aprenda mejor).
    """
    # p_d = p0 + np.array([-1.7*np.sin(0.3*t), 0.0, -0.5])
    # v_d = np.array([-1.7*0.3*np.cos(0.3*t), 0.0, -0.0])
    # a_d = np.array([1.7*(0.3**2)*np.sin(0.3*t), 0.0, -0.0])
    p_d = p0 + np.array([-1.7*np.sin(0.3*t), 0.0, -0.5])
    v_d = np.array([-1.7*0.3*np.cos(0.3*t), 0.0, -0.0])
    a_d = np.array([1.7*(0.3**2)*np.sin(0.3*t), 0.0, -0.0])

    yaw_d = 0.0
    yawdot_d = 0.0
    return p_d, v_d, a_d, yaw_d, yawdot_d


# ============================================================
# 4) Features φ (polinomiales) como el paper (simplificado)
#    Paper usa (x,z, xdot, zdot) en el plano P. :contentReference[oaicite:6]{index=6}
# ============================================================

def phi_polynomial(pos, vel, n=2):
    """
    Construye φ = [x^n,...,x, z^n,...,z, vx^n,...,vx, vz^n,...,vz, 1]
    con n=2 por defecto.

    Esto es fácil, pequeño y funciona bien para empezar.
    """
    x = pos[0]
    z = pos[2]
    vx = vel[0]
    vz = vel[2]

    # Potencias descendentes: n, n-1, ..., 1
    px = [x**k for k in range(n, 0, -1)]
    pz = [z**k for k in range(n, 0, -1)]
    pvx = [vx**k for k in range(n, 0, -1)]
    pvz = [vz**k for k in range(n, 0, -1)]

    # + bias 1
    phi = np.array(px + pz + pvx + pvz + [1.0], dtype=float)
    return phi


# ============================================================
# 5) RLS continuo (paper):  Wdot = P φ ε^T,  Pdot = λP - P φ φ^T P
#    Inicialización: W=0, P=I. :contentReference[oaicite:7]{index=7}
# ============================================================

class ContinuousRLS:
    def __init__(self, dim_phi, forgetting_lambda=0.1):
        """
        dim_phi: dimensión de φ
        forgetting_lambda: λ del paper (factor de olvido en Pdot)
        """
        self.dim_phi = dim_phi
        self.lam = forgetting_lambda

        # W_hat ∈ R^{dim_phi x 3} (para estimar fuerza 3D)
        self.W = np.zeros((dim_phi, 3))

        # P ∈ R^{dim_phi x dim_phi}
        self.P = np.eye(dim_phi)

    def predict_force(self, phi):
        """f_hat = W^T φ  -> vector (3,)"""
        return self.W.T @ phi

    def update(self, phi, eps, dt):
        """
        phi: (dim_phi,)
        eps: (3,) error de fuerza
        dt: paso de integración
        """
        phi = phi.reshape(-1, 1)      # (dim_phi,1)
        eps = eps.reshape(1, 3)       # (1,3)

        # Wdot = P φ ε^T
        Wdot = self.P @ phi @ eps

        # Pdot = λP - P φ φ^T P
        Pdot = self.lam * self.P - (self.P @ phi @ phi.T @ self.P)

        # Integración Euler (simple)
        self.W += Wdot * dt
        self.P += Pdot * dt


# ============================================================
# 6) Mellinger (actitud) para "realizar" una fuerza deseada u (mundo)
# ============================================================

def mellinger_attitude_from_force(R, omega_body, u_world, yaw_d, yawdot_d, J):
    """
    Dado:
      - R actual
      - omega_body actual
      - u_world: fuerza deseada total en mundo (N)
    Retorna:
      - thrust (N)
      - torque M (N*m) en body
    """
    # Ganancias actitud (tuneables)
    KR = np.diag([8.0, 8.0, 2.0])
    Kw = np.diag([0.25, 0.25, 0.15])

    # (A) Construir R_des desde u_world y yaw_d
    u_norm = np.linalg.norm(u_world)
    if u_norm < 1e-6:
        u_norm = 1e-6

    b3_des = u_world / u_norm
    b1_c = np.array([np.cos(yaw_d), np.sin(yaw_d), 0.0])

    b2_des = np.cross(b3_des, b1_c)
    if np.linalg.norm(b2_des) < 1e-6:
        b2_des = np.array([0.0, 1.0, 0.0])
    else:
        b2_des /= np.linalg.norm(b2_des)

    b1_des = np.cross(b2_des, b3_des)
    R_des = np.column_stack((b1_des, b2_des, b3_des))

    # (B) thrust = u_world · b3_actual
    b3 = R[:, 2]
    thrust = float(u_world.dot(b3))

    # (C) error de actitud geométrico
    e_R_mat = 0.5 * (R_des.T @ R - R.T @ R_des)
    e_R = vee(e_R_mat)

    # (D) omega deseada (simplificada: solo yawdot)
    omega_des_world = np.array([0.0, 0.0, yawdot_d])
    omega_des_body = R.T @ omega_des_world

    e_omega = omega_body - omega_des_body

    # (E) torque: -KR e_R - Kw e_w + omega x (J omega)
    Jomega = J @ omega_body
    coriolis = np.cross(omega_body, Jomega)
    M = -KR @ e_R - Kw @ e_omega + coriolis

    return thrust, M


# ============================================================
# 7) Mixer simple (thrust + torques -> 4 motores)
# ============================================================

def simple_mixer(thrust, M, ctrl_min=0.0, ctrl_max=2.0):
    """
    Mezclador simple para tus 4 motores (ctrlrange 0..2).
    Ajusta escalas si no despega o vibra.
    """
    kT = 0.7
    kMx = 0.8
    kMy = 0.8
    kMz = 0.3

    uT = kT * thrust
    Mx, My, Mz = M

    u0 = uT - kMy*My - kMz*Mz
    u1 = uT + kMy*My + kMz*Mz
    u2 = uT + kMx*Mx - kMz*Mz
    u3 = uT - kMx*Mx + kMz*Mz

    ctrl = np.array([
        clamp(u0, ctrl_min, ctrl_max),
        clamp(u1, ctrl_min, ctrl_max),
        clamp(u2, ctrl_min, ctrl_max),
        clamp(u3, ctrl_min, ctrl_max),
    ], dtype=float)

    return ctrl


# ============================================================
# 8) Main: Control completo del paper (simplificado a 1 dron)
# ============================================================

def main():
    XML_PATH = "models/jiawei_model_rep_single.xml"  # <-- cambia al nombre real
    model = mujoco.MjModel.from_xml_path(XML_PATH)
    data = mujoco.MjData(model)

    dt = model.opt.timestep

    # Body del dron
    drone_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, "drone_3d")

    # Masa del dron (subtree)
    m = float(model.body_subtreemass[drone_id])

    # Inercia (aprox diagonal)
    J_diag = model.body_inertia[drone_id].copy()
    J = np.diag(J_diag)

    # Forward para inicializar datos
    mujoco.mj_forward(model, data)
    p0 = data.xpos[drone_id].copy()

    # -----------------------------
    # RLS: define n y crea estimador
    # -----------------------------
    n = 10  # grado polinomio
    dim_phi = (n*4 + 1)  # x,z,vx,vz (cada uno n términos) + bias
    rls = ContinuousRLS(dim_phi=dim_phi, forgetting_lambda=0.1)

    # -----------------------------
    # Para observer de aceleración: usamos diferencia finita
    # -----------------------------
    prev_vel = data.cvel[drone_id, 3:6].copy()
    acc_filt = np.zeros(3)
    alpha_acc = 0.3  # filtro 0..1 (más alto = menos filtrado)

    # -----------------------------
    # Ganancias kp, kd del paper para traslación (escala simple)
    # ui = kp e + kd e_dot + mg e3 + m a_d - f_hat  :contentReference[oaicite:8]{index=8}
    # -----------------------------
    kp = 6.0
    kd = 4.5

    g = 9.81
    e3 = np.array([0.0, 0.0, 1.0])

    # -----------------------------
    # Reloj para tiempo real (para que no vaya rápido)
    # -----------------------------
    next_time = time.time()
    t = 0.0

    with viewer.launch_passive(model, data) as v:
        while v.is_running():
            # ====================================================
            # (1) Leer estado del dron
            # ====================================================
            pos = data.xpos[drone_id].copy()

            omega_world = data.cvel[drone_id, 0:3].copy()
            vel_world   = data.cvel[drone_id, 3:6].copy()

            quat = data.xquat[drone_id].copy()
            R = quat_to_rotmat(quat)
            omega_body = R.T @ omega_world

            # ====================================================
            # (2) Aplicar fuerza desconocida REAL (solo simulación)
            # ====================================================
            f_unk = unknown_force_world(pos)
            data.xfrc_applied[drone_id, 0:3] = 0.0
            data.xfrc_applied[drone_id, 3:6] = f_unk

            # ====================================================
            # (3) Referencia deseada
            # ====================================================
            p_d, v_d, a_d, yaw_d, yawdot_d = desired_trajectory(t, p0)

            # Errores como en el paper:
            # e = p_d - p ; e_dot = v_d - v  :contentReference[oaicite:9]{index=9}
            e = p_d - pos
            e_dot = v_d - vel_world

            # ====================================================
            # (4) Predicción de fuerza externa con RLS: f_hat = W^T φ
            #     (en el paper se hace en el plano P, aquí mundo)
            # ====================================================
            phi = phi_polynomial(pos, vel_world, n=n)
            f_hat = rls.predict_force(phi)  # (3,)

            # ====================================================
            # (5) Control de traslación del paper:
            #     u = kp e + kd e_dot + mg e3 + m a_d - f_hat  :contentReference[oaicite:10]{index=10}
            # ====================================================
            u_world = kp * e + kd * e_dot + m * g * e3 + m * a_d - f_hat

            # ====================================================
            # (6) Convertir esa fuerza deseada a (thrust, torques)
            #     usando control geométrico (Mellinger)
            # ====================================================
            thrust, M = mellinger_attitude_from_force(
                R=R,
                omega_body=omega_body,
                u_world=u_world,
                yaw_d=yaw_d,
                yawdot_d=yawdot_d,
                J=J
            )

            # Mixer -> motores
            data.ctrl[:] = simple_mixer(thrust, M, ctrl_min=0.0, ctrl_max=2.0)

            # ====================================================
            # (7) Paso de simulación (esto actualiza vel, etc.)
            # ====================================================
            mujoco.mj_step(model, data)
            v.sync()

            # ====================================================
            # (8) OBSERVADOR de fuerza (paper):
            #     f^o = m p_ddot + mg e3 - u
            #     aquí estimamos p_ddot con diff de velocidad
            #     (paper eq. (6)) :contentReference[oaicite:11]{index=11}
            # ====================================================
            # a_meas (diff finita) + filtro
            acc_meas = (vel_world - prev_vel) / dt
            prev_vel = vel_world.copy()

            acc_filt = (1 - alpha_acc) * acc_filt + alpha_acc * acc_meas

            # u aplicado "real" aproximado:
            # en este diseño u_world era la fuerza deseada.
            # para un primer prototipo, usamos u_world como u aplicado (simple).
            # (si quieres más exactitud, podemos usar thrust*b3_actual)
            f_obs = m * acc_filt + m * g * e3 - u_world

            # ====================================================
            # (9) Error de estimación ε = f_obs - f_hat
            #     (en paper: ε = R_P^T f^o - f_hat_P ) :contentReference[oaicite:12]{index=12}
            # ====================================================
            eps = f_obs - f_hat

            # ====================================================
            # (10) Update RLS continuo:
            #      Wdot = P φ ε^T ; Pdot = λP - P φ φ^T P  :contentReference[oaicite:13]{index=13}
            # ====================================================
            rls.update(phi=phi, eps=eps, dt=dt)

            # ====================================================
            # (11) Tiempo + “tiempo real” (no correr muy rápido)
            # ====================================================
            t += dt
            next_time += dt
            time.sleep(max(0.0, next_time - time.time()))


if __name__ == "__main__":
    main()


C:\Users\edwar\AppData\Local\Temp\ipykernel_27804\3330403007.py:140: RuntimeWarning: overflow encountered in matmul
  Pdot = self.lam * self.P - (self.P @ phi @ phi.T @ self.P)
C:\Users\edwar\AppData\Local\Temp\ipykernel_27804\3330403007.py:137: RuntimeWarning: invalid value encountered in matmul
  Wdot = self.P @ phi @ eps
C:\Users\edwar\AppData\Local\Temp\ipykernel_27804\3330403007.py:140: RuntimeWarning: invalid value encountered in matmul
  Pdot = self.lam * self.P - (self.P @ phi @ phi.T @ self.P)
